In [7]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np

from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook


import xlrd
import os

In [2]:
def clean_desempleo_empleo_mensual(path):
    try:
        archivos = pd.Series(os.listdir(path))
        dane_des_emp_mensual_nombre = archivos[archivos.str.contains('anexo_desestacionalizado_empleo')][0]

        for i in archivos:
            if i ==  dane_des_emp_mensual_nombre:
                try:
                    data = load_workbook(path+"\{}".format(i))
                    sheets = pd.Series(data.sheetnames).str.lower()
                    tnal_mensual_index = sheets[sheets.str.contains('tnal mensual')].index[0]
                except:
                    data = xlrd.open_workbook_xls(path+"\{}".format(i))
                    sheets = pd.Series(data.sheet_names()).str.lower()
                    tnal_mensual_index = sheets[sheets.str.contains('tnal mensual')].index[0]

                df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_mensual_index)

                ind = ['tgp','to','td','ocupados','desocupados','inactivos']
                series = pd.DataFrame({})
                for i in ind:
                    ser_index = df[df.applymap(lambda x: str(x).lower() == i)].dropna(how='all',axis=0).index[0]
                    ser = df.iloc[ser_index,1:].rename(i)
                    if i == 'ocupados' or i == 'desocupados' or i == 'inactivos':
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')*1000      
                    else:
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')

                    series[i] = ser
        
        series.to_csv(path+"\desempleo_desest_mensual.csv",sep=';',decimal=',')
        os.remove(path+'\{}'.format(dane_des_emp_mensual_nombre))
        
    except:
        print('El : {} no se pudo limpiar correctamente'.format(dane_des_emp_mensual_nombre))
        pass

In [17]:
def clean_desempleo_empleo_sexo(path):
    archivos = pd.Series(os.listdir(path))
    dane_sexo_nombre = archivos[archivos.str.contains('anexo_sexo_')][0]

    for i in archivos:
        if i == dane_sexo_nombre:
            try: 
                data = load_workbook(path+"\{}".format(i))
                sheets = data.sheetnames
            except:
                data = xlrd.open_workbook_xls(path+"\{}".format(i))
                sheets = data.sheet_names()

            sheets = pd.Series(sheets).str.lower().str.replace(' ','')
            tnal_nacional_sexo_index = sheets[sheets.str.contains('pytn')].index[0]

            df = pd.read_excel(path+"\{}".format(i),sheet_name=tnal_nacional_sexo_index)


            ind = [i.lower().replace(' ','_') for i in pd.Series("""% población en edad de trabajar 
TGP
TO
TD
T.D. Abierto
T.D. Oculto
Población total
Población en edad de trabajar
Población económicamente activa
Ocupados
Desocupados
Abiertos
Ocultos
Inactivos""").str.split('\n')[0]]


            series_tnac = pd.DataFrame({})
            series_hombres = pd.DataFrame({})
            series_mujeres = pd.DataFrame({})

            for i in ind:

                # Total Nacional
                ser_index_nac = df[df.applymap(lambda x: str(x).lower().replace(' ','_') == i)].dropna(how='all',axis=0).index[0]

                if ser_index_nac:
                    ser = df.iloc[ser_index_nac,1:].rename(i) 
                    if i == 'tgp' or i== 'to' or i == 'td' or i == 't.d._abierto' or i == 't.d._oculto':
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')
                    else:
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')*1000
                    series_tnac[i] = ser

                #Hombres
                ser_index_hom = df[df.applymap(lambda x: str(x).lower().replace(' ','_') == i)].dropna(how='all',axis=0).index[1]

                if ser_index_hom:
                    ser = df.iloc[ser_index_hom,1:].rename(i)  
                    if i == 'tgp' or i== 'to' or i == 'td' or i == 't.d._abierto' or i == 't.d._oculto':
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')
                    else:
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')*1000
                    series_hombres[i] = ser

                #Mujeres
                ser_index_muj = df[df.applymap(lambda x: str(x).lower().replace(' ','_') == i)].dropna(how='all',axis=0).index[2]

                if ser_index_muj:
                    ser = df.iloc[ser_index_muj,1:].rename(i)  
                    if i == 'tgp' or i== 'to' or i == 'td' or i == 't.d._abierto' or i == 't.d._oculto':
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')
                    else:
                        ser = ser.set_axis(pd.date_range(start='2001-01-01',periods=len(ser),freq='M')).astype('float')*1000
                    series_mujeres[i] = ser

    series_tnac.to_csv(path+"\sexo_tnac.csv",sep=';',decimal=',')
    series_hombres.to_csv(path+"\sexo_hombres.csv",sep=';',decimal=',')
    series_hombres.to_csv(path+"\sexo_mujeres.csv",sep=';',decimal=',')
    os.remove(path+"\{}".format(dane_sexo_nombre))

In [4]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

import urllib3
import csv

In [68]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_experimental_option("prefs", {"download.default_directory":r"D:\Desktop\HUB\mercado_laboral", "download.prompt_for_download": False,
"download.directory_upgrade": True,
"safebrowsing.enabled": True}) 

#Excepciones
ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

###########################################################
################ OJO #####################################

# Deben descargar el chromedirver.exe en https://chromedriver.chromium.org/ si su navegador prederminado es chrome
driver_path = "C:/Users/Laptop/HUB_DAMAC/chromedriver.exe" #corresponde a la carpeta en la que guardo el archivo
driver = webdriver.Chrome(driver_path, options=options)

driver.get("https://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/segun-sexo/mercado-laboral-historicos")

try:
    WebDriverWait(driver,5)\
    .until(EC.element_to_be_clickable((By.LINK_TEXT,"Anexos")))\
    .click()
except:
    print('Hubo un problema al descargar el desempleo desestacionalizado ')


In [18]:
clean_desempleo_empleo_sexo(r"D:\Desktop\HUB\mercado_laboral")

In [21]:
from limpieza import limpieza_mercado_laboral

In [22]:
l = limpieza_mercado_laboral()

****
___

In [1]:
from HUB import HUB_DAMAC

In [2]:
laboral = HUB_DAMAC.mercado_laboral()

In [3]:
laboral.fuente_laboral

,Indicador,Frecuencia,Fuente
0,Tasa de desempleo,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
1,Tasa de ocupación,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
2,Tasa global de participación,Mensual,https://totoro.banrep.gov.co/estadisticas-econ...
3,Informalidad,Trimestral,https://www.dane.gov.co/index.php/estadisticas...
4,Desempleo desestacionalizado,Mensual,https://www.dane.gov.co/index.php/estadisticas...
5,Desempleo por sexo,Trimestre móvil,https://www.dane.gov.co/index.php/estadisticas...


In [4]:
laboral.actualizar(carpeta=r"D:\Desktop\HUB\mercado_laboral",actualizar_todo=False,
                   indicadores=['Desempleo desestacionalizado',"Desempleo por sexo"],
                   excel = True,hipervinculos=False)

C:\Programas\Miniconda\envs\hubdamac\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
